<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="fig/cover-small.jpg">

*本文摘自 Jake VanderPlas 的 [Python 之旅](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp )；内容可在 [GitHub](https://github.com/jakevdp/WhirlwindTourOfPython) 上找到。*

*文本和代码根据 [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE) 许可证发布；另请参阅配套项目，[Python 数据科学手册](https://github.com/jakevdp/PythonDataScienceHandbook)。*

*中文翻译由 [ZhangCongke](https://ckeyzhang.github.io/) 提供，项目可在 [GitHub](https://github.com/CKeyZhang/WhirlwindTourOfPython-CN) 上找到。*


<!--NAVIGATION-->
< [错误和异常](09-Errors-and-Exceptions.ipynb) | [目录](Index.ipynb) | [列表推导式](11-List-Comprehensions.ipynb) >

# 迭代器

在数据分析中，一个重要的环节通常是重复进行类似的计算，一遍又一遍，以实现自动化。
例如，你可能有一个名字列表，想要将其拆分为名字和姓氏，或者你可能有一系列日期，想要将它们转换为某种标准格式。
Python 的一个解决方案就是 *迭代器* 语法。
我们已经见过 `range` 迭代器了：

In [25]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

接下来我们将深入探究。
事实证明，在 Python 3 中，`range` 并不是一个列表，而是一个 *迭代器* ，了解它的运作原理对于掌握 Python 中一系列非常实用的功能至关重要。

## 迭代列表
迭代器或许通过具体地迭代列表来最容易理解。
考虑以下代码：

In [1]:
for value in [2, 4, 6, 8, 10]:
    # 执行某些操作
    print(value + 1, end=' ')

3 5 7 9 11 

熟悉的 "``for x in y``" 语法允许我们对列表中的每个值重复执行某些操作。
事实与表面现象并不相同。
当你写下类似 "``for val in L``" 的代码时，Python 解释器会检查它是否有迭代器接口，你可以通过内置的 `iter` 函数自行检查：

In [2]:
iter([2, 4, 6, 8, 10])

正是这个迭代器对象提供了 `for` 循环所需的运行机制。
迭代器对象是一个容器，只要有效，它就能让你访问下一个对象，这可以通过内置函数 `next` 来查看：

In [3]:
I = iter([2, 4, 6, 8, 10])

In [4]:
print(next(I))

2


In [5]:
print(next(I))

4


In [6]:
print(next(I))

6


这种间接性的作用是什么？
事实证明，这非常有用，因为它允许 Python 将不是列表的东西当作列表来对待。

## ``range()``：列表并不总是列表
或许这种间接迭代最常见的例子是 Python 3 中的 ``range()`` 函数（在 Python 2 中名为 ``xrange()``），它返回的不是列表，而是一个特殊的 ``range()`` 对象：

In [7]:
range(10)

range(0, 10)

``range``，像列表一样，也暴露了一个迭代器：

In [8]:
iter(range(10))

所以 Python 知道要像对待列表一样对待它：

In [9]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

迭代器间接性的好处是 *从未真正创建完整的列表！*
我们可以通过执行一个范围计算来验证这一点，如果实际创建了列表，这将耗尽我们的系统内存（注意，在 Python 2 中，``range`` 创建一个列表，所以运行以下代码将不会得到好结果！）：

In [10]:
N = 10 ** 12
for i in range(N):
    if i >= 10: break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

如果 ``range`` 真正创建了那个包含一万亿个值的列表，它将占用数十太字节的机器内存：这是一种浪费，因为我们忽略了除前 10 个值之外的所有值！

事实上，没有任何理由迭代器必须在某个时刻结束！
Python 的 ``itertools`` 库包含一个 ``count`` 函数，它就像一个无限的范围：

In [11]:
from itertools import count

for i in count():
    if i >= 10:
        break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

如果我们没有在这里插入循环中断，它将愉快地一直计数，直到手动中断或终止进程（例如，使用 ``ctrl-C``）。

## 有用的迭代器
这种迭代器语法几乎普遍用于 Python 内置类型以及我们将在后续部分探讨的更具体的数据科学对象。
在这里，我们将介绍 Python 语言中一些更有用的迭代器：

### ``enumerate``
很多时候，你需要在迭代数组中的值的同时，还要跟踪索引。
你可能会像这样操作：

In [12]:
L = [2, 4, 6, 8, 10]
for i in range(len(L)):
    print(i, L[i])

0 2
1 4
2 6
3 8
4 10


尽管这种方法可以奏效，但 Python 提供了一种更简洁的语法，使用 ``enumerate`` 迭代器：

In [13]:
for i, val in enumerate(L):
    print(i, val)

0 2
1 4
2 6
3 8
4 10


这是迭代列表索引和值的更“Pythonic”的方式。

### ``zip``
其他时候，你可能有多个列表，想要同时迭代它们。
你当然可以像前面非 Pythonic 的例子中那样迭代索引，但使用 ``zip`` 迭代器会更好，它可以将多个可迭代对象组合在一起：

In [14]:
L = [2, 4, 6, 8, 10]
R = [3, 6, 9, 12, 15]
for lval, rval in zip(L, R):
    print(lval, rval)

2 3
4 6
6 9
8 12
10 15


可以将任意数量的可迭代对象组合在一起，如果它们的长度不同，则以最短的长度为准。

### ``map`` 和 ``filter``
``map`` 迭代器接受一个函数，并将其应用于迭代器中的值：

In [15]:
# 找出前 10 个平方数
square = lambda x: x ** 2
for val in map(square, range(10)):
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 

``filter`` 迭代器看起来类似，不过它只通过那些使过滤函数返回 True 的值：

In [16]:
# 找出小于 10 的值，其中 x % 2 等于零
is_even = lambda x: x % 2 == 0
for val in filter(is_even, range(10)):
    print(val, end=' ')

0 2 4 6 8 

``map`` 和 ``filter`` 函数，连同 Python 的 ``functools`` 模块中的 ``reduce`` 函数，是 *函数式编程* 风格的基本组成部分，尽管 *函数式编程* 并不是 Python 世界中的主要编程风格，但它也有其坚定的支持者（例如，[pytoolz](https://toolz.readthedocs.org/en/latest/) 库）。

### 迭代器作为函数参数

我们在 [``*args`` 和 ``**kwargs``：灵活的参数](#*args-和-**kwargs：灵活的参数) 中看到，``*args`` 和 ``**kwargs`` 可以用来将序列和字典传递给函数。
事实证明，``*args`` 语法不仅适用于序列，还适用于任何迭代器：

In [17]:
print(*range(10))

0 1 2 3 4 5 6 7 8 9


因此，例如，我们可以巧妙地将前面的 ``map`` 示例压缩为以下代码：

In [18]:
print(*map(lambda x: x ** 2, range(10)))

0 1 4 9 16 25 36 49 64 81


使用这个技巧可以让我们回答 Python 学习者论坛上经常出现的一个问题：为什么没有与 ``zip()`` 相反的 ``unzip()`` 函数？
如果你把自己关在黑暗的房间里好好思考一番，你可能会意识到，``zip()`` 的反面就是... ``zip()``！关键在于 ``zip()`` 可以将任意数量的迭代器或序列组合在一起。观察：

In [19]:
L1 = (1, 2, 3, 4)
L2 = ('a', 'b', 'c', 'd')

In [20]:
z = zip(L1, L2)
print(*z)

(1, 'a') (2, 'b') (3, 'c') (4, 'd')


In [21]:
z = zip(L1, L2)
new_L1, new_L2 = zip(*z)
print(new_L1, new_L2)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


好好思考这个问题。如果你明白了它的原理，那你对 Python 迭代器的理解就已经相当深入了！

## 专用迭代器：``itertools``

我们简要地了解了无限 ``range`` 迭代器，``itertools.count``。
``itertools`` 模块包含了许多有用的迭代器；花时间探索这个模块，看看有哪些可用的工具是很值得的。
例如，考虑 ``itertools.permutations`` 函数，它迭代一个序列的所有排列：

In [22]:
from itertools import permutations
p = permutations(range(3))
print(*p)

(0, 1, 2) (0, 2, 1) (1, 0, 2) (1, 2, 0) (2, 0, 1) (2, 1, 0)


类似地，``itertools.combinations`` 函数迭代列表中所有独特的 ``N`` 个值的组合：

In [23]:
from itertools import combinations
c = combinations(range(4), 2)
print(*c)

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)


与之有些相关的是 ``product`` 迭代器，它迭代两个或多个可迭代对象之间所有成对的集合：

In [24]:
from itertools import product
p = product('ab', range(3))
print(*p)

('a', 0) ('a', 1) ('a', 2) ('b', 0) ('b', 1) ('b', 2)


``itertools`` 中还有许多其他有用的迭代器：完整的列表以及一些示例可以在 Python 的 [在线文档](https://docs.python.org/3.5/library/itertools.html) 中找到。

<!--NAVIGATION-->
< [错误和异常](09-Errors-and-Exceptions.ipynb) | [目录](Index.ipynb) | [列表推导式](11-List-Comprehensions.ipynb) >